In [68]:
import os
import arcpy
from samgeo import SamGeo2, regularize
import sys

# 取得目前開啟的 ArcGIS Pro 專案
aprx = arcpy.mp.ArcGISProject("CURRENT")

print("專案路徑：", aprx.filePath)              # .aprx 檔案完整路徑
print("預設 gdb：", aprx.defaultGeodatabase)   # 預設的 geodatabase
print("專案資料夾：", aprx.homeFolder)         # 專案所在的資料夾 (home folder)

# 取得目前活動的 Map (通常就是你 ArcGIS Pro 視窗左邊的那個)
m = aprx.activeMap

print("目前 Map 名稱:", m.name)

sr = m.spatialReference
print("圖層空間參考:", sr.name, sr.factoryCode)

# 檢查工作環境路徑與python版本
print(sys.executable)
print("Conda 環境路徑:", sys.prefix)

專案路徑： D:\ArcGIS\AI_training\AI_training.aprx
預設 gdb： D:\ArcGIS\AI_training\AI_training.gdb
專案資料夾： D:\ArcGIS\AI_training
目前 Map 名稱: KBay
圖層空間參考: WGS_1984_UTM_Zone_4N 32604
C:\Program Files\ArcGIS\Pro\bin\ArcGISPro.exe
Conda 環境路徑: C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo


In [143]:
# ---- 1) 輸入與輸出路徑設定 ----
ortho = r"D:\SfM_coral_reefs_product\KaneoheBay\2025\k2b\ortho_splits\ortho_split_04.tif"                 # 你的正射影像
points_fc = r"D:\ArcGIS\AI_training\split_points.gdb\T4"       # 你的點圖層（可含 label 欄位）
label_field = "label"                            # 若無此欄位，程式會預設所有點為 1（正點）

out_workspace = r"D:\ArcGIS\AI_training\AI_training.gdb"                # 結果輸出 GDB

# 取得座標數量
count = int(arcpy.management.GetCount(points_fc)[0])
print(f"共有 {count} 個點")

# 轉成 [ [x, y], ... ] 格式
point_coords_batch = []
with arcpy.da.SearchCursor(points_fc, ["SHAPE@X", "SHAPE@Y"], where_clause="Shape IS NOT NULL") as cur:
    for x, y in cur:
        point_coords_batch.append([x, y])

# 印出結果（前幾筆檢查）
print("example of point_coords_batch =", point_coords_batch[:1])

# 取得 SpatialReference 物件
sr = arcpy.Describe(points_fc).spatialReference
# EPSG 代碼
epsg = sr.factoryCode
if epsg and int(epsg) > 0:
    point_crs = f"EPSG:{int(epsg)}"
else:
    # 如果沒有 EPSG，則輸出 WKT 字串
    point_crs = sr.exportToString()
print("座標系名稱:", sr.name)
print("EPSG 代碼:", epsg)
print("SAM2 要用的 point_crs:", point_crs)

共有 138 個點
example of point_coords_batch = [[624604.6014, 2373480.101399999]]
座標系名稱: WGS_1984_UTM_Zone_4N
EPSG 代碼: 32604
SAM2 要用的 point_crs: EPSG:32604


In [144]:
sam = SamGeo2(model_id="sam2-hiera-small", automatic=False)

In [145]:
# 指定影像
sam.set_image(ortho)

C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [146]:
# 進行點提示分割，輸出二值遮罩（uint8；1=物件，0=背景）
sam.predict_by_points(
    point_coords_batch=point_coords_batch,     # ✅ 直接丟 coords，不要 [coords]
    point_crs= point_crs,
    output="mask_4.tif",
    dtype="uint8",
)

C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [147]:
workspace = arcpy.env.workspace
in_raster = os.path.join(workspace, "mask_4.tif")

# 指定 File GDB
gdb_path = os.path.join(workspace, "geosam_output.gdb")

# 設定輸出 Feature Class 名稱（不需要 .shp）
out_fc = os.path.join(gdb_path, "sam_shpraw_Z4")

# 執行轉換
arcpy.conversion.RasterToPolygon(in_raster, out_fc)

<Result 'D:\\ArcGIS\\AI_training\\geosam_output.gdb\\sam2_mask_Z4'>

In [148]:
arcpy.env.overwriteOutput = True

# 各自的 GDB 路徑
poly_gdb  = r"D:\ArcGIS\AI_training\geosam_output.gdb"   # 多邊形所在 gdb
point_gdb = r"D:\ArcGIS\AI_training\split_points.gdb"    # 點所在 gdb
out_gdb   = r"D:\ArcGIS\AI_training\shp_filtered.gdb"    # 輸出 gdb

# 各 Feature Class 完整路徑
polys_path  = os.path.join(poly_gdb,  "sam_shpraw_Z4")       # 多邊形 FC
points_path = os.path.join(point_gdb, "T4")                  # 點 FC
out_fc      = os.path.join(out_gdb,   "sam_filtered_Z4")     # 輸出 FC（不要 .shp）

# 若輸出 gdb 不存在就建立
if not arcpy.Exists(out_gdb):
    arcpy.management.CreateFileGDB(os.path.dirname(out_gdb),
                                   os.path.splitext(os.path.basename(out_gdb))[0])

# --- 1) Spatial Join：只保留多邊形欄位 ---
fm = arcpy.FieldMappings()
fm.addTable(polys_path)  # 只加入多邊形欄位，不加入點欄位

arcpy.analysis.SpatialJoin(
    target_features=polys_path,
    join_features=points_path,
    out_feature_class=out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",      # 只保留與點相交的多邊形
    field_mapping=fm,             # 關鍵：不帶入點的欄位
    match_option="CONTAINS"      # 嚴格包含可改 "CONTAINS_CLEMENTINI"
)

# --- 2) 找出最大面積要素的 OID（用 SHAPE@AREA，不用新增欄位） ---
oid_field = arcpy.Describe(out_fc).oidFieldName
max_oid, max_area = None, float("-inf")
with arcpy.da.SearchCursor(out_fc, [oid_field, "SHAPE@AREA"]) as cur:
    for oid, area in cur:
        if area is not None and area > max_area:
            max_area = area
            max_oid = oid

# --- 3) 刪除：面積 < 門檻 + 面積最大者 ---
threshold = 0.00001  # 你的門檻；單位依圖層座標系而定
deleted_small = 0
deleted_max   = 0

with arcpy.da.UpdateCursor(out_fc, [oid_field, "SHAPE@AREA"]) as ucur:
    for oid, area in ucur:
        if area is None:
            continue
        # 先刪面積過小的
        if area < threshold:
            ucur.deleteRow()
            deleted_small += 1

# 再刪「最大面積」那一筆（若還存在）
if max_oid is not None:
    found_max = False
    with arcpy.da.UpdateCursor(out_fc, [oid_field]) as ucur2:
        for oid, in ucur2:
            if oid == max_oid:
                ucur2.deleteRow()
                deleted_max = 1
                found_max = True
                break
    # 可選：如果最大那筆剛好已被門檻刪掉，不需再做任何事
    if not found_max:
        deleted_max = 0

print(f"完成：輸出 {out_fc}，刪除小面積 {deleted_small} 筆、刪除最大面積 {deleted_max} 筆。")

完成：輸出 D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z4，刪除小面積 0 筆、刪除最大面積 1 筆。


In [149]:
out_fc   = r"D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z4"
out_fc_noholes = r"D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z4"

# 門檻（小於此面積的「洞」會被填掉）
hole_area_threshold = 0.0005  # 單位依圖層座標系統而定

arcpy.management.EliminatePolygonPart(
    in_features=out_fc,
    out_feature_class=out_fc_noholes,
    condition="AREA",                  # 以面積當條件；也可用 "PERCENT"
    part_area=hole_area_threshold,     # 面積門檻
    part_option="CONTAINED_ONLY"       # 只移除完全被外環包住的部分（即洞）
)

print(f"完成：小於 {hole_area_threshold} 單位² 的洞已填補 → {out_fc_noholes}")

完成：小於 0.0005 單位² 的洞已填補 → D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z4
